In [1]:
import pandas
import os
import sys
import requests
from matplotlib import pyplot 

In [2]:
HTSW = os.path.expanduser('~diane/proj/htsworkflow')
if HTSW not in sys.path:
    sys.path.append(HTSW)
from htsworkflow.submission.encoded import ENCODED

In [3]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [4]:
def lookup_experiment(experiment_id):
    experiment = server.get_json(experiment_id)
    for replicate in experiment['replicates']:
        library = replicate['library']
        if 'aliases' not in library:
            library = server.get_json(replicate['library'])

        print(experiment['accession'], library['accession'], library['aliases'], experiment['description'])

In [5]:
def parse_samtools_stats(record):
    columns = {
        '@id': str,
        'quality_metric_of': list,
        'duplicates': int,
        'duplicates_qc_failed': int, 
        'mapped': int, 
        'mapped_pct': float, 
        'mapped_qc_failed': int,
        'total': int, 
        'total_qc_failed': int,
    }
    results = {}
    for name, converter in columns.items():
        results[name] = converter(record[name])
    return results
    #return pandas.Series(result)


In [6]:
def parse_pct(value):
    assert value.endswith('%'), 'Expected % at end of {}'.format(value)
    return float(value[:-1])

def parse_star_stats(record):
    columns = {
        '@id': str,
        'quality_metric_of': list,
        '% of chimeric reads': parse_pct,
        '% of reads mapped to multiple loci': parse_pct,
        '% of reads mapped to too many loci': parse_pct,
        '% of reads unmapped: other': parse_pct,
        '% of reads unmapped: too many mismatches': parse_pct,
        '% of reads unmapped: too short': parse_pct,
        'Average input read length': float,
        'Average mapped length': float,
        'Deletion average length': float,
        'Deletion rate per base': parse_pct,
        'Insertion average length': float,
        'Insertion rate per base': parse_pct,
        'Mapping speed, Million of reads per hour': float,
        'Mismatch rate per base, %': parse_pct,
        'Number of chimeric reads': int,
        'Number of input reads': int,
        'Number of reads mapped to multiple loci': int,
        'Number of reads mapped to too many loci': int,
        'Number of splices: AT/AC': int,
        'Number of splices: Annotated (sjdb)': int,
        'Number of splices: GC/AG': int,
        'Number of splices: GT/AG': int,
        'Number of splices: Non-canonical': int,
        'Number of splices: Total': int,
        'Uniquely mapped reads %': parse_pct,
        'Uniquely mapped reads number': int,
    }
    results = {}
    for name, converter in columns.items():
        results[name] = converter(record[name])
        
    return results

In [7]:
def parse_gene_type_quantification(record):
    columns = {
        '@id': str,
        'quality_metric_of': list,
        'Mt_rRNA': int, 
        'antisense': int, 
        'miRNA': int, 
        'processed_transcript': int,
        'protein_coding': int, 
        'rRNA': int,
        'ribozyme': int, 
        'sRNA': int, 
        'scaRNA': int, 
        'sense_intronic': int, 
        'sense_overlapping': int, 
        'snRNA': int, 
        'snoRNA': int, 
        'spikein': int
    }
    results = {}
    for name, converter in columns.items():
        results[name] = converter(record[name])
    
    return results
    #return pandas.Series(result)

In [8]:
def parse_mad_metric(record):
    def get_href(record):
        return record['href']
    
    columns = {
        '@id': str,
        'quality_metric_of': list,
        'attachment': get_href,
        'SD of log ratios': float,
        'Pearson correlation': float,
        'Spearman correlation': float,
        'MAD of log ratios': float,
    }
    results = {}
    for name, converter in columns.items():
        results[name] = converter(record[name])    
    return results

In [9]:
def parse_genes_detected(record):
    columns = {
        '@id': str,
        'quality_metric_of': list,
        'number_of_genes_detected': int,
    }
    results = {}
    for name, converter in columns.items():
        results[name] = converter(record[name])
        
    return results    

In [10]:
def files_to_library(experiment, file_object):
    libraries = None
    library_aliases = None
    assert len(file_object['replicate_libraries']) == 1
    for file_library_id in file_object['replicate_libraries']:
        for experiment_replicate in experiment['replicates']:
            library = experiment_replicate['library']
            if library['@id'] == file_library_id:
                libraries = library['accession']
                library_aliases = library['aliases'][0]
    return (libraries, library_aliases)

def find_experiment_quality_metrics(experiment):   
    qc_report = {
        'SamtoolsFlagstatsQualityMetric': parse_samtools_stats,
        'StarQualityMetric': parse_star_stats,
        'GeneTypeQuantificationQualityMetric': parse_gene_type_quantification,
        'MadQualityMetric': parse_mad_metric,
        'GeneQuantificationQualityMetric': parse_genes_detected,
    }

    library_map = {}
    library_aliases_map = {}
    for f in experiment['files']:
        libraries, aliases = files_to_library(experiment, f)
        library_map[f['@id']] = libraries
        library_aliases_map[f['@id']] = aliases
        
    reports = {}
    for f in experiment['files']:
        if 'quality_metrics' in f:
            for metric in f['quality_metrics']:
                metric_type = metric['@type'][0]
                value = qc_report[metric_type](metric)
                for_libraries = tuple([library_aliases_map[x] for x in value['quality_metric_of']])
                reports.setdefault(metric_type, {})[for_libraries] = value
    return reports

def format_alias(alias):
    return ",".join([x.split(':')[1] for x in alias])

def report_experiment(experiment):
    print(experiment['accession'], experiment['description'])
    reports = find_experiment_quality_metrics(experiment)
    
    star_quality = {}
    for library_alias in reports['StarQualityMetric']:
        formatted_alias = format_alias(library_alias)
        star_qc = reports['StarQualityMetric'][library_alias]
        mapped = star_qc['Number of reads mapped to multiple loci'] + star_qc['Uniquely mapped reads number']
        fraction_mapped = mapped / star_qc['Number of input reads']
        star_quality[formatted_alias] = {
            'multi': star_qc['Number of reads mapped to multiple loci'],
            'uniq': star_qc['Uniquely mapped reads number'],
            'mapped': mapped,
            'total': star_qc['Number of input reads'],
            '%mapped': "{:.4}".format(fraction_mapped * 100),
        }
    print(pandas.DataFrame(star_quality).T)
            

    if 'MadQualityMetric' in reports:
        mad = reports['MadQualityMetric']
        for library_alias in mad:
            formatted_alias = format_alias(library_alias)
            print(formatted_alias, 'Spearman', mad[library_alias]['Spearman correlation'])    
    else:
        print('No correlation for {}'.format(library_alias))

    gene_type = {}
    for library_alias in reports['GeneTypeQuantificationQualityMetric']:
        formatted_alias = format_alias(library_alias)
        gene_type_counts = reports['GeneTypeQuantificationQualityMetric'][library_alias]
        del gene_type_counts['@id']
        del gene_type_counts['quality_metric_of']
        gene_type[formatted_alias] = gene_type_counts
        
    gene_type = pandas.DataFrame(gene_type)
    f = pyplot.figure(figsize=(8,8))
    ax = f.add_subplot(1,1,1)
    gene_type.T.plot.bar(stacked=True, ax=ax)
    #return gene_type   
    #reports['GeneQuantificationQualityMetric']

In [11]:
#query = "https://www.encodeproject.org/search/?type=Experiment&@id=%2Fexperiments%2FENCSR379DEC%2F&@id=%2Fexperiments%2FENCSR877FRY%2F&@id=%2Fexperiments%2FENCSR631FXT%2F&@id=%2Fexperiments%2FENCSR306IAW%2F&@id=%2Fexperiments%2FENCSR336VTK%2F&@id=%2Fexperiments%2FENCSR429EGC%2F"
#query = "https://www.encodeproject.org/search/?type=Experiment&@id=%2Fexperiments%2FENCSR479MNN%2F&@id=%2Fexperiments%2FENCSR288RRZ%2F&@id=%2Fexperiments%2FENCSR899OKE%2F&@id=%2Fexperiments%2FENCSR464VSR%2F&@id=%2Fexperiments%2FENCSR774MGO%2F&@id=%2Fexperiments%2FENCSR129VBC%2F&@id=%2Fexperiments%2FENCSR420YFF%2F&@id=%2Fexperiments%2FENCSR942YMN%2F&@id=%2Fexperiments%2FENCSR648YUM%2F&@id=%2Fexperiments%2FENCSR903XMI%2F&@id=%2Fexperiments%2FENCSR244HHV%2F&@id=%2Fexperiments%2FENCSR168PXI%2F&@id=%2Fexperiments%2FENCSR308XAR%2F&@id=%2Fexperiments%2FENCSR484WZL%2F"
query = "https://www.encodeproject.org/search/?type=Experiment&@id=%2Fexperiments%2FENCSR479MNN%2F&@id=%2Fexperiments%2FENCSR288RRZ%2F&@id=%2Fexperiments%2FENCSR899OKE%2F&@id=%2Fexperiments%2FENCSR464VSR%2F&@id=%2Fexperiments%2FENCSR774MGO%2F&@id=%2Fexperiments%2FENCSR129VBC%2F&@id=%2Fexperiments%2FENCSR420YFF%2F&@id=%2Fexperiments%2FENCSR942YMN%2F&@id=%2Fexperiments%2FENCSR648YUM%2F&@id=%2Fexperiments%2FENCSR903XMI%2F&@id=%2Fexperiments%2FENCSR244HHV%2F&@id=%2Fexperiments%2FENCSR168PXI%2F&@id=%2Fexperiments%2FENCSR308XAR%2F&@id=%2Fexperiments%2FENCSR484WZL%2F&@id=%2Fexperiments%2FENCSR538FRP%2F"
search = server.get_json(query)
for row in search['@graph']:
    experiment = server.get_json(row['@id'])
    report_experiment(experiment)

ENCSR288RRZ Stam Placenta 11005


KeyError: 'replicate_libraries'